In [1]:
import pandas as pd
import os
import scipy.stats as stats


In [2]:
carpetas = ['GUROBI','CPLEX','MLM','ESGH','LPDH','Versiones']

In [3]:
gur = []
mlm = []
esgh = []
lpdh = []
cplex = []
versiones = []

for c in carpetas:
    archivos = os.listdir(c)
    for ar in archivos: 
        # print(sorted(ar.rstrip(".xlsx").split("_")))
        # print(sorted(ar.rstrip(".xlsx").split("_")))
        tiempo, solver, capacidad, nodos  = sorted(ar.rstrip(".xlsx").split("_"))
        tiempo = int(tiempo)
        capacidad = int(capacidad[1:])
        nodos = int(nodos[1:])
        datos = pd.read_excel(f"{c}/{ar}")
        datos["Q"] = capacidad
        datos["nodos"] = nodos
        datos["tiempos"] = tiempo
        if solver == "GUROBI":
            gur.append(datos)
        if solver == "CPLEX":
            cplex.append(datos)
        elif solver == "MLM":
            mlm.append(datos)
        elif solver == "ESGH":
            esgh.append(datos)
        elif solver == "LPDH":
            lpdh.append(datos)
        elif solver in ["Expv1", "Expv2", "Expv3", "Expv4"]:
            datos["version"] = "Version" + solver[-1]
            versiones.append(datos)
        else: pass
        
gurobis = pd.concat(gur)
cplexs  = pd.concat(cplex)
mlms    = pd.concat(mlm)
esghs   = pd.concat(esgh)
lpdhs   = pd.concat(lpdh)
versiones   = pd.concat(versiones)

In [4]:
#gurobis[["LB","UB"]] = gurobis[["LB","UB"]].round(2)
#cplexs[["LB","UB"]] = cplexs[["LB","UB"]].round(2)
#mlms[["min", "avg"]] = mlms[["min", "avg"]].round(2)
#versiones[["min", "avg"]] = versiones[["min", "avg"]].round(2)

In [5]:
bks = pd.concat((
    mlms.groupby(["name", "Q", "nodos"]).agg(bks = ("min", "min")).reset_index(), 
    gurobis.groupby(["name", "Q", "nodos"]).agg(bks = ("UB", "min")).reset_index(),
    cplexs.groupby(["name", "Q", "nodos"]).agg(bks = ("UB", "min")).reset_index(),
    esghs.groupby(["name", "Q", "nodos"]).agg(bks = ("cost", "min")).reset_index(),
    lpdhs.groupby(["name", "Q", "nodos"]).agg(bks = ("cost", "min")).reset_index(),
    versiones.groupby(["name", "Q", "nodos"]).agg(bks = ("min", "min")).reset_index(),
))

bks = bks.groupby(["name", "Q", "nodos"]).agg(bks = ("bks", "min")).reset_index()

gurobis = gurobis.merge(bks, on = ["name", "Q", "nodos"])
cplexs  = cplexs.merge(bks, on = ["name", "Q", "nodos"] )
mlms    = mlms.merge(bks, on = ["name", "Q", "nodos"])
esghs   = esghs.merge(bks, on = ["name", "Q", "nodos"])
lpdhs   = lpdhs.merge(bks, on = ["name", "Q", "nodos"])
versiones = versiones.merge(bks, on = ["name", "Q", "nodos"])

In [6]:
gurobis["best"] = gurobis["UB"]
cplexs["best"] = cplexs["UB"]
mlms["best"] = mlms["min"]
versiones["best"] = versiones["min"]
lpdhs["best"] = lpdhs["cost"]
esghs["best"] = esghs["cost"]

gurobis["gap_best"]    = (gurobis["UB"] - gurobis["bks"]) / gurobis["bks"]
cplexs["gap_best"]     = (cplexs["UB"] - cplexs["bks"]) / cplexs["bks"]
mlms["gap_best"]       = (mlms["min"] - mlms["bks"]) / mlms["bks"]
mlms["gap_avg"]        = (mlms["avg"] - mlms["bks"]) / mlms["bks"]
versiones["gap_best"]  = (versiones["min"] - versiones["bks"]) / versiones["bks"]
versiones["gap_avg"]   = (versiones["avg"] - versiones["bks"]) / versiones["bks"]
lpdhs["gap"]           = (lpdhs["cost"] - lpdhs["bks"]) / lpdhs["bks"]
esghs["gap"]           = (esghs["cost"] - esghs["bks"]) / esghs["bks"]

gurobis["hit"]      =( gurobis["best"] - gurobis["bks"]).apply(lambda x: 1 if x == 0 else 0) 
mlms["hit"]         =( mlms["best"] - mlms["bks"]).apply(lambda x: 1 if x == 0 else 0) 
versiones["hit"]    =( versiones["best"] - versiones["bks"]).apply(lambda x: 1 if x == 0 else 0) 
lpdhs["hit"]        =( lpdhs["best"] - lpdhs["bks"]).apply(lambda x: 1 if x == 0 else 0) 
esghs["hit"]        =( esghs["best"] - esghs["bks"]).apply(lambda x: 1 if x == 0 else 0) 

In [7]:
mlms_versiones = mlms[(mlms.tiempos == 180) & (mlms.nodos >= 150)]
mlms_versiones.loc[:,"version"] = "mlm"
versiones2 = pd.concat((versiones , mlms_versiones))
tabla_versiones = versiones2.groupby(["version"]).agg(gap_best  = ("gap_best", "mean"),
                                   gap_avg  =  ("gap_avg", "mean"),
                                   avg_time =  ("t_avg", "mean"),
                                   hit_sum  =  ("hit", "sum")).reset_index()

tabla_versiones

C:\Users\rablo\AppData\Local\Temp\ipykernel_25692\2738699161.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mlms_versiones.loc[:,"version"] = "mlm"


,version,gap_best,gap_avg,avg_time,hit_sum
0,Version1,0.048639,0.072654,180.009754,1
1,Version2,0.011909,0.020293,173.635559,63
2,Version3,0.009526,0.017002,173.841993,75
3,Version4,0.010503,0.018692,174.651170,72
4,mlm,0.011642,0.021039,174.242555,64


In [8]:
bks["name"] = bks["name"].str.upper()
bks["fullname"] = bks["name"] + "-" + bks["Q"].astype("str") + "-" + bks["nodos"].astype("str") 
dict_bks = dict()
for i, row in bks.iterrows():
    dict_bks[row["fullname"]] = row["bks"]

In [9]:
import json
with open('bks.json', 'w') as f:
    json.dump(dict_bks, f)

In [10]:
#name = "C101-10-100"
#with open("bks.json","r") as f:
#    bks = json.loads(f.read())[name]
#    

In [12]:
bks

,name,Q,nodos,bks,fullname
0,C101,5,100,829.318065,C101-5-100
1,C101,10,100,563.366659,C101-10-100
2,C101,15,100,528.277571,C101-15-100
3,C101,20,100,488.999400,C101-20-100
4,C101,1000,100,466.520000,C101-1000-100
...,...,...,...,...,...
875,RC2_2_9,15,200,1754.590637,RC2_2_9-15-200
876,RC2_2_9,20,150,1335.576500,RC2_2_9-20-150
877,RC2_2_9,20,200,1616.801441,RC2_2_9-20-200
878,RC2_2_9,1000,150,1175.827444,RC2_2_9-1000-150


In [19]:
mlms[(mlms.nodos == 100) & (mlms.Q >=20)].sort_values("gap_best", ascending = False).head(50)

,name,min,avg,t_avg,Q,nodos,tiempos,bks,best,gap_best,gap_avg,hit
2010,RC103,638.953726,645.799957,60.095336,20,100,60,633.316363,638.953726,8.901338e-03,1.971147e-02,0
1986,R206,633.573819,635.495137,60.000477,20,100,60,628.239226,633.573819,8.491340e-03,1.154960e-02,0
1842,RC103,611.395827,614.285969,60.063292,1000,100,60,606.707671,611.395827,7.727208e-03,1.249086e-02,0
2021,R203,623.335725,627.894144,60.000455,20,100,60,619.158923,623.335725,6.745927e-03,1.410820e-02,0
1996,R111,613.492927,613.492927,60.077301,20,100,60,609.485624,613.492927,6.574893e-03,6.574893e-03,0
1982,RC107,631.448899,631.448899,60.051127,20,100,60,627.944210,631.448899,5.581211e-03,5.581211e-03,0
2001,R106,656.458715,658.002497,60.013287,20,100,60,653.416782,656.458715,4.655426e-03,7.018055e-03,0
2017,C204,545.123847,545.244089,60.030675,20,100,60,543.044262,545.123847,3.829494e-03,4.050917e-03,0
2023,R104,596.230112,596.230112,60.045986,20,100,60,594.430658,596.230112,3.027189e-03,3.027189e-03,0
2013,R107,622.042073,629.709851,60.000596,20,100,60,620.360490,622.042073,2.710655e-03,1.507085e-02,0


In [24]:
versiones[(versiones.nodos == 150) & (versiones.version == "Version4")].sort_values("gap_avg", ascending = False).head(50)

,name,min,avg,t_avg,Q,nodos,tiempos,version,bks,best,gap_best,gap_avg,hit
51,r1_2_8,1233.754936,1273.665597,180.148770,1000,150,180,Version4,1211.106432,1233.754936,0.018701,0.051655,0
1143,c2_2_4,1273.742963,1286.637711,180.002857,15,150,180,Version4,1227.086285,1273.742963,0.038022,0.048531,0
1159,c2_2_9,1330.161940,1350.669310,180.070467,15,150,180,Version4,1292.334569,1330.161940,0.029271,0.045139,0
2143,r1_2_3,2310.045034,2342.065443,180.000863,5,150,180,Version4,2247.278565,2310.045034,0.027930,0.042179,0
2155,c2_2_3,2115.716931,2155.340629,180.001326,5,150,180,Version4,2068.368930,2115.716931,0.022891,0.042048,0
2035,r1_2_4,2268.336814,2286.847254,180.005974,5,150,180,Version4,2196.361462,2268.336814,0.032770,0.041198,0
2019,c2_2_2,2177.426458,2207.571535,180.003533,5,150,180,Version4,2120.476141,2177.426458,0.026857,0.041074,0
1939,r2_2_8,2307.516579,2328.796537,180.003239,5,150,180,Version4,2238.488807,2307.516579,0.030837,0.040343,0
2159,rc2_2_2,2351.781429,2381.198726,180.002771,5,150,180,Version4,2291.106252,2351.781429,0.026483,0.039323,0
19,r2_2_8,1201.151396,1213.260890,180.776601,1000,150,180,Version4,1169.774307,1201.151396,0.026823,0.037175,0
